## Inverse Cooking: Recipe Generation from Food Images

In [56]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from args import get_parser
import pickle
from model import get_model
from torchvision import transforms
from utils.output_utils import prepare_output
from PIL import Image
import time
from shutil import copyfile
import csv

Set ```data_dir``` to the path including vocabularies and model checkpoint

In [57]:
########move data to sample data folder

selected_files = []
rootdir = '/home/ct2020dl5787/VireoFood172/ready_chinese_food'
for subdir, dirs, files in os.walk(rootdir):
    
    if(len(files) > 0):
        for i in range(10) :
            file_name = sorted(files)[i]
            prefix = subdir.split("/")[-1]
            selected_files.append(prefix + "/" + file_name)

print(len(selected_files))


1720


In [13]:
# copydir = '/home/ct2020dl5787/VireoFood172/sample_pictures1720'
# for eachfile in selected_files:
#     file_name = eachfile.replace("/", "_")
#     ori = os.path.join(rootdir, eachfile)
#     dst = os.path.join(copydir, file_name)
#     copyfile(ori, dst)
    
##############################

In [58]:
vocab_file = '/home/ct2020dl5787/VireoFood172/vocab172_mapping.csv'
chinese_vocab = []
mapped_vocab = []
with open(vocab_file) as file:
    csv_reader = csv.reader(file, delimiter=',')
    for row in csv_reader:
        #check duplicate
        chinese_vocab.append(row[0])
        mapped_vocab.append(row[1])
        #print("originally: ", row[0], " mapped to ", row[1])

In [59]:
true_labels = dict()
ing_labels_file = open('/home/ct2020dl5787/VireoFood172/SplitAndIngreLabel/IngreLabel.txt', 'r')
contents = ing_labels_file.readlines()
for line in contents:
    filename = line.split()[0]
    labels = line.split()[1:]
    true_labels[filename] = labels
ing_labels_file.close()

In [60]:
img_names = []
img_ingre = []
img_chinese_ingre = []
for f in selected_files:
    name = "/"+f
    if name not in true_labels:
        print("error")
    else :
        binary_label = true_labels[name]
        mapped_labels = []
        mapped_chinese_labels = []
        for index, label in enumerate(binary_label):
            if(label == "1") :
                mapped_labels.append(mapped_vocab[index])
                mapped_chinese_labels.append(chinese_vocab[index])
        
        img_ingre.append(mapped_labels)
        img_chinese_ingre.append(mapped_chinese_labels)
        img_names.append(f.replace("/", "_"))

In [61]:

img_dict = {'img_names': img_names, 'true_ingredients': img_ingre, 'ingredients_Chinese' : img_chinese_ingre}  
    
true_label_df = pd.DataFrame(img_dict) 


In [62]:
true_label_df

,img_names,true_ingredients,ingredients_Chinese
0,92_075811n2tr1trtvr21vvao.jpg,"[scallion, pepper, beef]","[Minced green onion, Shredded pepper, Shredded..."
1,92_075817xlk22xxxagbk296x.jpg,"[scallion, pepper, beef]","[Minced green onion, Shredded pepper, Shredded..."
2,92_100053030.1.jpg,"[scallion, pepper, beef]","[Minced green onion, Shredded pepper, Shredded..."
3,92_10_0.jpg,"[Parsley, beef]","[Chinese Parsleycoriander, Shredded beef tripe]"
4,92_10_1.jpg,"[Parsley, chili, garlic, beef]","[Chinese Parsleycoriander, Crushed hot and dry..."
...,...,...,...
1715,63_10_14.jpg,"[tofu, chili]","[Tofu chunks, Chili oil]"
1716,63_10_16.jpg,"[scallion, tofu, chili]","[Minced green onion, Tofu chunks, Chili oil]"
1717,63_10_18.jpg,"[scallion, pork, tofu, chili]","[Minced green onion, Minced pork, Tofu chunks,..."
1718,63_10_19.jpg,"[scallion, pepper, tofu]","[Minced green onion, Hot and dry pepper, Tofu ..."


In [64]:

vocab_with_end = list(set(chinese_vocab))
vocab_with_end.insert(0, '<end>')

In [73]:
chinese2idx = dict()
idx2chinese = dict()
for index, word in enumerate(vocab_with_end):
    chinese2idx[word] = index
    idx2chinese[index] = word

In [76]:
#create label vectors 
tensor_matrix = np.zeros((1720, 20, len(chinese2idx)))

In [77]:
tensor_matrix.shape

(1720, 20, 353)

In [78]:
for i in range(len(img_names)):
    for j in range(len(img_chinese_ingre[i])):
        tensor_matrix[i][j][chinese2idx[img_chinese_ingre[i][j]]] = 1      
        

In [79]:
all_ingredient_lists = []
for i in range(tensor_matrix.shape[0]):
    ing = []
    for j in range(tensor_matrix.shape[1]):
        if( 1 in tensor_matrix[i][j]):
            ing.append(idx2chinese[np.where(tensor_matrix[i][j]==1)[0][0]])
             
    all_ingredient_lists.append(ing)   
print(all_ingredient_lists)

[['Minced green onion', 'Shredded pepper', 'Shredded beef tripe'], ['Minced green onion', 'Shredded pepper', 'Shredded beef tripe'], ['Minced green onion', 'Shredded pepper', 'Shredded beef tripe'], ['Chinese Parsleycoriander', 'Shredded beef tripe'], ['Chinese Parsleycoriander', 'Crushed hot and dry chili', 'Crushed garlic', 'Shredded beef tripe'], ['Chinese Parsleycoriander', 'Julienned cucumber', 'Shredded beef tripe'], ['Pepper slices', 'Shredded beef tripe'], ['Chinese Parsleycoriander', 'Shredded beef tripe'], ['Crushed pepper', 'Shredded beef tripe', 'Celery'], ['Crushed pepper', 'Shredded beef tripe', 'Celery'], ['Black sesame', 'Yam chunks', 'Crystal sugar'], ['Black sesame', 'Cherry tomato slices', 'Cucumber slices', 'Yam chunks', 'Crystal sugar'], ['Yam chunks', 'Crystal sugar'], ['Yam chunks', 'Crystal sugar'], ['Yam chunks', 'Crystal sugar'], ['Black sesame', 'Cherry tomato slices', 'Cucumber slices', 'Yam chunks', 'Crystal sugar'], ['Yam chunks', 'Parsley', 'Crystal sugar

In [82]:
with open('/home/ct2020dl5787/VireoFood172/pickle_data/onehot1720matrix.pkl','wb') as f:
    pickle.dump(tensor_matrix, f)
with open('/home/ct2020dl5787/VireoFood172/pickle_data/onehot1720matrix_vocab.pkl','wb') as f:
    pickle.dump(chinese2idx, f)

with open('/home/ct2020dl5787/VireoFood172/pickle_data/onehot1720matrix_idx2word.pkl','wb') as f:
    pickle.dump(idx2chinese, f)

In [83]:
with open('/home/ct2020dl5787/VireoFood172/pickle_data/onehot1720matrix_vocab.pkl','rb') as f:
    vocab = pickle.load(f)

In [84]:
vocab

{'<end>': 0,
 'Orange slices': 1,
 'Whole boiled chicken': 2,
 'Steamed bread': 3,
 'kidney bean': 4,
 'Boild eggs': 5,
 'Lentinus edodes slices': 6,
 'Minced pork': 7,
 'Shredded pig ears': 8,
 'Crushed pepper': 9,
 'Vermicelli': 10,
 'White sesame': 11,
 'Squid pieces': 12,
 'Okra slices': 13,
 'Sea cucumber': 14,
 'Stewed Pork': 15,
 'Radix Astragali': 16,
 'Crispy sausage': 17,
 'Pleurotus ostreatus': 18,
 'Preserved egg chunks': 19,
 'Scrambled egg': 20,
 'Snow peas': 21,
 'Green vegetables': 22,
 'Cabbage': 23,
 'Boiled chicken slices': 24,
 'Seared green onion': 25,
 'shumai': 26,
 'Strawberry': 27,
 'Fried bread stick': 28,
 'Beans': 29,
 'Tofu chunks': 30,
 'Brunoise diced onion': 31,
 'Shredded kelp': 32,
 'Eggplant': 33,
 'Scallion pancake': 34,
 'Cured meat chunks': 35,
 'Stinky tofu': 36,
 'Preserved vegetables': 37,
 'Crab': 38,
 'Lotus root chunks': 39,
 'Julienned ham': 40,
 'Soya-bean milk': 41,
 'Chili powder': 42,
 'Black sesame': 43,
 'Garlic clove': 44,
 'Whole chi

In [21]:
true_label_df.to_pickle('/home/ct2020dl5787/VireoFood172/pickle_data/sample1720.pkl')

In [10]:
# code will run in gpu if available and if the flag is set to True, else it will run on cpu
use_gpu = True
device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
map_loc = None if torch.cuda.is_available() and use_gpu else 'cpu'


# device = torch.device('cpu')
# map_loc = None


In [11]:
print(device)

cuda


In [12]:
# code below was used to save vocab files so that they can be loaded without Vocabulary class
#ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_ingrs.pkl'), 'rb'))
#ingrs_vocab = [min(w, key=len) if not isinstance(w, str) else w for w in ingrs_vocab.idx2word.values()]
#vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_toks.pkl'), 'rb')).idx2word
#pickle.dump(ingrs_vocab, open('../demo/ingr_vocab.pkl', 'wb'))
#pickle.dump(vocab, open('../demo/instr_vocab.pkl', 'wb'))
data_dir = ""
ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'ingr_vocab.pkl'), 'rb'))
vocab = pickle.load(open(os.path.join(data_dir, 'instr_vocab.pkl'), 'rb'))

ingr_vocab_size = len(ingrs_vocab)
instrs_vocab_size = len(vocab)
output_dim = instrs_vocab_size

In [13]:
#print(ingrs_vocab)

In [14]:
print (instrs_vocab_size, ingr_vocab_size)

23231 1488


In [15]:
t = time.time()
import sys; sys.argv=['']; del sys
args = get_parser()
args.maxseqlen = 15
args.ingrs_only=False
model = get_model(args, ingr_vocab_size, instrs_vocab_size)
# Load the trained model parameters
model_path = os.path.join('/home/ct2020dl5787/inversecooking/model/checkpoints', 'modelbest.ckpt')
model.load_state_dict(torch.load(model_path, map_location=map_loc))
model.to(device)
model.eval()
model.ingrs_only = False
model.recipe_only = False
model.reduction = 'none'
print ('loaded model')
print ("Elapsed time:", time.time() -t)


/home/ct2020dl5787/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


loaded model
Elapsed time: 5.803600788116455


In [16]:
transf_list_batch = []
transf_list_batch.append(transforms.ToTensor())
transf_list_batch.append(transforms.Normalize((0.485, 0.456, 0.406), 
                                              (0.229, 0.224, 0.225)))
to_input_transf = transforms.Compose(transf_list_batch)

In [17]:
#greedy = [True, False, False, False]
greedy = [True]
#beam = [-1, -1, -1, -1]
beam = [-1]
temperature = 1.0
numgens = len(greedy)

Set ```use_urls = True``` to get recipes for images in ```demo_urls```. 

You can also set ```use_urls = False``` and get recipes for images in the path in ```data_dir/test_imgs```.

In [18]:
import requests
from io import BytesIO
import random
from collections import Counter
unpickled_df = pd.read_pickle("df_recipe1023.pkl")
use_urls = False # set to true to load images from demo_urls instead of those in test_imgs folder
show_anyways = False #if True, it will show the recipe even if it's not valid
image_folder = os.path.join('/home/ct2020dl5787/VireoFood172/sample_pictures1720')
#TODO1:load data
if not use_urls:
    demo_imgs = os.listdir(image_folder)
    random.shuffle(demo_imgs)

demo_urls = ['https://food.fnr.sndimg.com/content/dam/images/food/fullset/2013/12/9/0/FNK_Cheesecake_s4x3.jpg.rend.hgtvcom.826.620.suffix/1387411272847.jpeg',
         'https://www.196flavors.com/wp-content/uploads/2014/10/california-roll-3-FP.jpg']
demo_urls = unpickled_df["url"].tolist()
demo_files = demo_urls if use_urls else demo_imgs

demo_files = demo_files

In [19]:
for img_file in demo_files:
    if not img_file.endswith(".jpg"):
        print(img_file)

layer1.json
layer2.json


In [20]:
info_all = []
err_pic = []
start = time.time()
for img_file in demo_files:
    
    if not use_urls and img_file.endswith(".jpg"):
#         response = requests.get(img_file)
#         try:
#             image = Image.open(BytesIO(response.content))
        image_path = os.path.join(image_folder, img_file)
        image = Image.open(image_path).convert('RGB')

        transf_list = []
        transf_list.append(transforms.Resize(256))
        transf_list.append(transforms.CenterCrop(224))
        transform = transforms.Compose(transf_list)

        image_transf = transform(image)
        image_tensor = to_input_transf(image_transf).unsqueeze(0).to(device)

       # plt.imshow(image_transf)
       # plt.axis('off')
      #  plt.show()
      #  plt.close()

        num_valid = 1

        generate_ing = []
        generate_recipt = []
        generate_title = []
        generate_score = []
        for i in range(numgens):
            with torch.no_grad():
                outputs = model.sample(image_tensor, greedy=greedy[i],temperature=temperature, beam=beam[i], true_ingrs=None)
            ingr_ids = outputs['ingr_ids'].cpu().numpy()
            recipe_ids = outputs['recipe_ids'].cpu().numpy()

            outs, valid = prepare_output(recipe_ids[0], ingr_ids[0], ingrs_vocab, vocab)
            #TODO2: Save output  
            generate_ing.append(outs['ingrs'])
            generate_recipt.append(outs['recipe'])
            generate_title.append(outs['title'])
            generate_score.append([valid['is_valid'],valid['score']])

#             if valid['is_valid'] or show_anyways:

#                 print ('RECIPE', num_valid)
#                 num_valid+=1
#                 #print ("greedy:", greedy[i], "beam:", beam[i])

#                 BOLD = '\033[1m'
#                 END = '\033[0m'
#                 print (BOLD + '\nTitle:' + END,outs['title'])

#                 print (BOLD + '\nIngredients:'+ END)
#                 print (', '.join(outs['ingrs']))

#                 print (BOLD + '\nInstructions:'+END)
#                 print ('-'+'\n-'.join(outs['recipe']))

#                 print ('='*20)

#             else:
#                 pass
#                 print ("Not a valid recipe!")
#                 print ("Reason: ", valid['reason'])
        info_all.append([img_file,generate_ing, generate_recipt,generate_title,generate_score])
    #except:
     
        #pass
        
print(start-time.time())
    

-5693.93252158165


In [21]:
generate_data = pd.DataFrame(info_all,columns = ['img_names','generate_ingre','generate_reci','generate_title','generate_score'])
all_info = pd.merge(true_label_df, generate_data , how='inner', on=['img_names'])

In [22]:
all_info

,img_names,true_ingredients,ingredients_Chinese,generate_ingre,generate_reci,generate_title,generate_score
0,92_075811n2tr1trtvr21vvao.jpg,"[scallion, pepper, beef]","[Minced green onion, Shredded pepper, Shredded...","[[pepper, oil, onion, clove, soy_sauce, sugar,...",[[Heat oil in a large skillet over medium heat...,[Spicy bean stir-fry],"[[True, 0.5]]"
1,92_075817xlk22xxxagbk296x.jpg,"[scallion, pepper, beef]","[Minced green onion, Shredded pepper, Shredded...","[[pepper, oil, onion, clove, beans, salt, toma...",[[Heat oil in a large skillet over medium heat...,[Green beans with tomatoes and onions],"[[True, 0.5555555555555556]]"
2,92_100053030.1.jpg,"[scallion, pepper, beef]","[Minced green onion, Shredded pepper, Shredded...","[[pepper, onion, oil, chicken, mushroom, salt,...",[[Heat oil in a large skillet over medium heat...,[Chicken and mushroom stir fry],"[[True, 0.4772727272727273]]"
3,92_10_0.jpg,"[Parsley, beef]","[Chinese Parsleycoriander, Shredded beef tripe]","[[pepper, oil, salt, potato, clove, onion, par...",[[Preheat oven to 400 degrees f (200 degrees c...,[Roasted red potatoes],"[[True, 0.6071428571428571]]"
4,92_10_1.jpg,"[Parsley, chili, garlic, beef]","[Chinese Parsleycoriander, Crushed hot and dry...","[[oil, onion, pepper, soy_sauce, sugar, chicke...","[[Mix all ingredients together., Serve on lett...",[Chicken salad],"[[True, 0.8]]"
...,...,...,...,...,...,...,...
1715,63_10_14.jpg,"[tofu, chili]","[Tofu chunks, Chili oil]","[[onion, pepper, oil, chicken, clove, tomato, ...","[[Heat oil in a large pot over medium heat., C...",[Chicken and tomato soup],"[[True, 0.48]]"
1716,63_10_16.jpg,"[scallion, tofu, chili]","[Minced green onion, Tofu chunks, Chili oil]","[[onion, pepper, oil, soy_sauce, clove, broth,...",[[Heat oil in a large skillet over medium-high...,[Spicy korean spicy chicken],"[[True, 0.46551724137931033]]"
1717,63_10_18.jpg,"[scallion, pork, tofu, chili]","[Minced green onion, Minced pork, Tofu chunks,...","[[onion, pepper, potato, clove, oil, tomato, s...","[[Heat oil in a large pot over medium heat., A...",[Caldo de pollo],"[[True, 0.573170731707317]]"
1718,63_10_19.jpg,"[scallion, pepper, tofu]","[Minced green onion, Hot and dry pepper, Tofu ...","[[onion, oil, soy_sauce, pepper, chicken, wate...","[[Cut the chicken into bite-sized pieces., Cut...",[Chicken and shimeji mushroom stir fry],"[[True, 0.4827586206896552]]"


In [23]:
all_info.to_pickle('chinese1720sample_allinfo.pkl')

In [ ]:
#TODO3: Evaluation

In [24]:
generate_data.to_pickle('output/generate_1720sample.pkl')

In [24]:
all_info = pd.read_pickle('chinese1720sample_allinfo.pkl')

In [28]:
print(all_info[all_info["img_names"] =="101_10_20.jpg"]["generate_reci"])

558    [[Boil the water and add the soup stock granul...
Name: generate_reci, dtype: object


In [33]:
all_info.loc[558,:]

img_names                                                  101_10_20.jpg
true_ingredients                                [noodles, water, wonton]
ingredients_Chinese                             [Noodles, Water, Wonton]
generate_ingre               [[water, soy_sauce, noodles, sugar, stock]]
generate_reci          [[Boil the water and add the soup stock granul...
generate_title                                [Easy and delicious ramen]
generate_score                              [[True, 0.5319148936170213]]
Name: 558, dtype: object

In [41]:
all_info[all_info["img_names"] =="105_1089358.1.jpg"]


,img_names,true_ingredients,ingredients_Chinese,generate_ingre,generate_reci,generate_title,generate_score
17,105_1089358.1.jpg,"[yam, sugar]","[Yam chunks, Crystal sugar]","[[sugar, butter, cinnamon, potato, salt]]",[[Preheat oven to 350 degrees f (175 degrees c...,[Sweet potato chips],"[[True, 0.7014925373134329]]"


In [43]:
all_info[all_info["img_names"] =="109_10_0.jpg"]

,img_names,true_ingredients,ingredients_Chinese,generate_ingre,generate_reci,generate_title,generate_score
1043,109_10_0.jpg,"[scallion, Parsley, duck]","[Minced green onion, Chinese Parsleycoriander,...","[[oil, pepper, onion, clove, salt, butter, par...","[[Preheat oven to 425 degrees f., In a large b...",[Balsamic roasted red onions],"[[False, 0.43333333333333335]]"


In [122]:
def get_ingred_f1(pred, label):
    '''
    input: 
        pred: a list of predicted ingredients
        label: a list of label ingredients
    output: 
        F-1 score of the prediction
    
    i.e.  
    in: 
        pred = ["tomato", "sugar", "beef"]
        label = ["potato", "tomato"]
     
    out: 
        0.4
    '''
    intersection = list(set(pred) & set(label))
    precision = len(intersection) / len(pred)
    recall = len(intersection) / len(label)
    f1 = 2 * precision * recall / (precision + recall)
    return f1


def get_ingred_IOU(pred, label):
    '''
    input: 
        pred: a list of predicted ingredients
        label: a list of label ingredients
    output: 
        IOU of the prediction
    
    i.e.  
    in: 
        pred = ["tomato", "sugar", "beef"]
        label = ["potato", "tomato"]
     
    out: 
        0.25
    '''
    intersection = len(list(set(pred) & set(label)))
    union = len(list(set(pred) | set(label)))
    iou = intersection / union
    return iou


import nltk
import string
def get_bleu_n_score(pred, label, n = 4):
    
    '''
    TODO: STEM not added
    input: 
        pred: One string of predict recipe 
        label: One string of reference recipe
        n(optional): up to n-gram.
    output: 
        bleu score
    
    i.e.  
    in: 
        pred = "Add the buttter"
        label = "Add half butter and mix well"
    out: 
        0.25
    '''
    weights = [1/n] * n
    pred_list = pred.translate(str.maketrans('', '', string.punctuation)).split()
    label_list = label.translate(str.maketrans('', '', string.punctuation)).split()
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([label_list], pred_list, weights)
    return BLEUscore



from nltk.translate.meteor_score import meteor_score
def get_meteor_score(pred, label):
    return meteor_score([label], pred)